# Item Response Models
- categories: [Julia, Turing, ItemResponse]

> Important: This is just a draft! I haven't 1) finished creating reasonable data sets for each model or 2) made even one pretty picture :dizzy_face:!

In [23]:
#collapse
using Turing
using Bijectors
using Gadfly
using DataFrames, DataFramesMeta
Gadfly.set_default_plot_size(900px, 300px)

## Item Response

Item response models are used to simultaneously make inferences about two interacting populations. Commonly, a population of test questions and a population of test takers (students) with the result (success/failure) of each student on each question they've seen. This is an interesting problem in that even in the basic case:

- students have different levels of aptitude
- questions have different levels of difficulty
- not every student sees every question
- not every question needs to be seen by the same number of students
- we should be able to make relative inferences between students (resp. questions) that have no overlapping questions (resp. students)
- the data is nonetheless fairly simple: `[correct (Boolean), student_id (categorical), question_id (categorical]`

I love these models because they're easy to extend in an intuitive way. I'm going to add a few random bells and whistles to the most vanilla version, and if you're interested the [Stan user guide](https://mc-stan.org/docs/2_18/stan-users-guide/item-response-models-section.html) has some good content on this topic and many others.

# Some IRT Models

## Vanilla Item-Response (aka 1PL)

For each student $s$, we have an aptitude $\alpha_s$ and for each question $q$ we have a difficulty $\gamma_q$. The likelihood of a correct response is informed by the difference between these two quantities:

$$\begin{aligned}
\alpha_s &\sim \mathrm{Normal}(0,5)\\
\gamma_q &\sim \mathrm{Normal}(0,5)\\
\beta_{s, q} &= \mathrm{logit}^{-1}(\alpha_s - \gamma_q)\\
\mathrm{correct_{s,q}} &\sim \mathrm{Bernoulli}(\beta_{s,q})\\
\end{aligned}$$

In [24]:
logit = bijector(Beta())   # bijection:  (0, 1)  →  ℝ
inv_logit = inv(logit)     # bijection:       ℝ  →  (0, 1)

student = [1,1,1,1,2,2,2,2,3,3,3,3]
question = [1,2,3,4,2,3,4,5,3,4,5,1]
correct = [
    true, true, true, false, 
    true, false, false, true, 
    false, false, false, true];

Some observations on the toy data:
- Everyone got question 1 correct (expect this to be rated as low difficulty)
- Everyone got question 4 wrong (high difficulty)
- Student 1 got all tested questions correct except question 4
- Student 3 got all tested questions incorrect except question 1
- Question 5 was only seen by student 3 (incorrect)

So, here's the model set up in Turing, and the result of the sampler below.

In [25]:
@model function irt_1pl(correct::Array{Bool}, student::Array{Int64}, question::Array{Int64})
    aptitude = Vector(undef, maximum(student))
    difficulty = Vector(undef, maximum(question))
    
    # priors
    for i in 1:length(aptitude)
        aptitude[i] ~ Normal(0,5)
    end
    for i in 1:length(difficulty)
        difficulty[i] ~ Normal(0,5)
    end        
    
    β = Vector(undef, length(correct))
    for i in 1:length(correct)
        β[i] = aptitude[student[i]] - difficulty[question[i]]
        correct[i] ~ Bernoulli(inv_logit(β[i]))
    end
end;

# Settings of the Hamiltonian Monte Carlo (HMC) sampler.
iterations = 1000
ϵ = 0.05
τ = 10;

irt_1pl_ch = sample(
    irt_1pl(correct, student, question), 
    HMC(ϵ, τ), iterations, 
    progress=true, drop_warmup=true)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (1000×17×1 Array{Float64,3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = aptitude[1], aptitude[2], aptitude[3], difficulty[1], difficulty[2], difficulty[3], difficulty[4], difficulty[5]
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, n_steps, nom_step_size, step_size

Summary Statistics
     parameters      mean       std   naive_se      mcse       ess      rhat 
         Symbol   Float64   Float64    Float64   Float64   Float64   Float64 

    aptitude[1]    3.2740    1.9350     0.0612    0.3207   22.7341    1.0402
    aptitude[2]    0.9977    2.2009     0.0696    0.5345    5.0679    1.2607
    aptitude[3]   -2.7975    3.1851     0.1007    0.9097    3.2885    1.5668
  difficulty[1]   -6.6429    4.3688     0.1382    1.3718    2.4382    2.1207
  difficulty[2]   -3.1028    2.2644     0.0716    0.5146   13.6893    1.0655
  difficulty[3]

Interesting, the only surprise for me is that I expected a wider spread for `difficulty[5]`, but otherwise looks very reasonable!

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (1000×17×1 Array{Float64,3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = aptitude[1], aptitude[2], aptitude[3], difficulty[1], difficulty[2], difficulty[3], difficulty[4], difficulty[5]
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, n_steps, nom_step_size, step_size

Summary Statistics
     parameters      mean       std   naive_se      mcse       ess      rhat 
         Symbol   Float64   Float64    Float64   Float64   Float64   Float64 

    aptitude[1]    4.0629    2.2346     0.0707    0.5976    5.6320    1.1870
    aptitude[2]    2.0359    2.2032     0.0697    0.5355    7.5336    1.1946
    aptitude[3]   -2.9905    2.4569     0.0777    0.6584    4.7843    1.3258
  difficulty[1]   -6.8570    2.6949     0.0852    0.6485    4.6609    1.3814
  difficulty[2]   -1.2865    2.9363     0.0929    0.7521   12.8605    1.0007
  difficulty[3]

## Question Quality (aka 2PL)

The purpose of asking questions is to probe the aptitude of the test taker, and some questions will do a much better job of guaranteeing a minimum skill level given a successful response. This is called "discrimination". Intuitively, a highly discriminating question would magnify the difference between a student's ability and the question's difficulty, so that both

- students with sufficient aptitude are more likely to succeed
- students with insufficient aptitude are more likely to fail

We can see that $\eta$ will accomplish this in the model below:

$$\begin{aligned}
\alpha_s &\sim \mathrm{Normal}(0,5)\\
\gamma_q &\sim \mathrm{Normal}(0,5)\\
\eta_q &\sim \mathrm{LogNormal}(0,2)\\
\beta_{s, q} &= \mathrm{logit}^{-1}(\eta_q * (\alpha_s - \gamma_q))\\
\mathrm{correct_{s,q}} &\sim \mathrm{Bernoulli}(\beta_{s,q})\\
\end{aligned}$$

In [26]:
@model function irt_2pl(correct::Array{Bool}, student::Array{Int64}, question::Array{Int64})
    aptitude = Vector(undef, maximum(student))
    difficulty = Vector(undef, maximum(question))
    discr = Vector(undef, maximum(question))
    
    # priors
    for i in 1:length(aptitude)
        aptitude[i] ~ Normal(0,5)
    end
    for i in 1:length(difficulty)
        difficulty[i] ~ Normal(0,5)
    end        
    for i in 1:length(difficulty)
        discr[i] ~ LogNormal(0,2)
    end        
    
    β = Vector(undef, length(correct))
    for i in 1:length(correct)
        β[i] = discr[question[i]] * (aptitude[student[i]] - difficulty[question[i]])
        correct[i] ~ Bernoulli(inv_logit(β[i]))
    end
end;

In [27]:
#collapse
irt_2pl_ch = sample(
    irt_2pl(correct, student, question), 
    HMC(ϵ, τ), iterations, 
    progress=true, drop_warmup=true)

Sampling:  37%|███████████████▏                         |  ETA: 0:00:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/brad/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
Sampling:  38%|███████████████▋                         |  ETA: 0:00:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC /Users/brad/.julia/packages/AdvancedHMC/MIxdK/src/hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:03


Chains MCMC chain (1000×22×1 Array{Float64,3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = aptitude[1], aptitude[2], aptitude[3], difficulty[1], difficulty[2], difficulty[3], difficulty[4], difficulty[5], discr[1], discr[2], discr[3], discr[4], discr[5]
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, n_steps, nom_step_size, step_size

Summary Statistics
     parameters      mean       std   naive_se      mcse       ess      rhat 
         Symbol   Float64   Float64    Float64   Float64   Float64   Float64 

    aptitude[1]    1.2991    2.5842     0.0817    0.6109   15.4744    1.0188
    aptitude[2]   -0.4991    2.5806     0.0816    0.7655    4.9059    1.1926
    aptitude[3]   -1.8909    3.3642     0.1064    0.9387    6.3762    1.1386
  difficulty[1]   -2.1490    3.1013     0.0981    0.8999    3.7623    1.3383
  difficulty[2]   -3.7273    1.9332     0.

## Guessing Behavior

It's common knowledge that guessing is advantageous on the SATs if you can eliminate at least 1 answer. This is because there are usually 5 responses and an incorrect response is penalized by 1/4 point. In the previous examples we assumed that question difficulty and student aptitude accounted for a span of possible $P(\mathrm{correct})$ covering $(0,1)$, but if the test taker can opportunistically guess (ie on a multiple choice test) then the true probabilities have some other lower bound, $(\delta, 1), \delta > 0$.

Modifying our first model to account for this is relatively straightforward:

$$\begin{aligned}
\delta &\sim \mathrm{Beta}(1, 2)\\
\alpha_s &\sim \mathrm{Normal}(0,5)\\
\gamma_q &\sim \mathrm{Normal}(0,5)\\
\beta_{s, q} &= \delta + (1-\delta)\mathrm{logit}^{-1}(\alpha_s - \gamma_q)\\
\mathrm{correct_{s,q}} &\sim \mathrm{Bernoulli}(\beta_{s,q})\\
\end{aligned}$$



In [28]:
@model function irt_guess(correct::Array{Bool}, student::Array{Int64}, question::Array{Int64})
    aptitude = Vector(undef, maximum(student))
    difficulty = Vector(undef, maximum(question))
    
    # priors
    for i in 1:length(aptitude)
        aptitude[i] ~ Normal(0,5)
    end
    for i in 1:length(difficulty)
        difficulty[i] ~ Normal(0,5)
    end
    guess_factor ~ Beta(1,2)
    
    β = Vector(undef, length(correct))
    for i in 1:length(correct)
        β[i] = aptitude[student[i]] - difficulty[question[i]]
        correct[i] ~ Bernoulli(guess_factor + (1-guess_factor)*inv_logit(β[i]))
    end
end;

In [29]:
#collapse
irt_guess_ch = sample(
    irt_guess(correct, student, question), 
    HMC(ϵ, τ), iterations, 
    progress=true, drop_warmup=true)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (1000×18×1 Array{Float64,3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = aptitude[1], aptitude[2], aptitude[3], difficulty[1], difficulty[2], difficulty[3], difficulty[4], difficulty[5], guess_factor
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, n_steps, nom_step_size, step_size

Summary Statistics
     parameters      mean       std   naive_se      mcse       ess      rhat 
         Symbol   Float64   Float64    Float64   Float64   Float64   Float64 

    aptitude[1]    2.4869    2.5707     0.0813    0.7223    3.2350    1.6621
    aptitude[2]    0.0467    2.1501     0.0680    0.4649    4.9934    1.3398
    aptitude[3]   -4.6788    2.3852     0.0754    0.5883   12.5814    1.0098
  difficulty[1]   -5.4882    3.0988     0.0980    0.7927   11.2340    1.0712
  difficulty[2]   -1.3732    1.8141     0.0574    0.3498   12.9492    1.1089
 

## Two Kinds of Questions

Students aren't universally adept at answering questions of different types, so let's add that to the model! For questions of type $t_i$ (ie $t(q)=t_i$), we apply the student's aptitude from that question type.

$$\begin{aligned}
\alpha_{s, t} &\sim \mathrm{Normal}(0,5)\\
\gamma_q &\sim \mathrm{Normal}(0,5)\\
\beta_{s, q, t} &= \mathrm{logit}^{-1}(\alpha_{s,t(q)} - \gamma_q)\\
\mathrm{correct_{s,q}} &\sim \mathrm{Bernoulli}(\beta_{s,q})\\
\end{aligned}$$

More fun (but maybe too much fun for this post :sweat_smile:) is that with multiple question types it would be pretty simple to bake in correlations in student aptitude across question types.

In [30]:
question_types = [1,2,1,2,1,2,1,2,1,2,1,2]

@model function irt_2types(
        correct::Array{Bool}, 
        student::Array{Int64}, 
        question::Array{Int64}, question_type::Array{Int64}
    )
    aptitude_1 = Vector(undef, maximum(student))
    aptitude_2 = Vector(undef, maximum(student))
    difficulty = Vector(undef, maximum(question))
    
    # priors
    for i in 1:length(aptitude_1)
        aptitude_1[i] ~ Normal(0,5)
        aptitude_2[i] ~ Normal(0,5)
    end
    for i in 1:length(difficulty)
        difficulty[i] ~ Normal(0,5)
    end
    
    β = Vector(undef, length(correct))
    for i in 1:length(correct)
        if question_type[i] == 1
            β[i] = aptitude_1[student[i]] - difficulty[question[i]]
        else
            β[i] = aptitude_2[student[i]] - difficulty[question[i]]
        end
        correct[i] ~ Bernoulli(inv_logit(β[i]))
    end
end;

In [31]:
#collapse
irt_2types_ch = sample(
    irt_2types(correct, student, question, question_types), 
    HMC(ϵ, τ), iterations, 
    progress=true, drop_warmup=true)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (1000×20×1 Array{Float64,3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = aptitude_1[1], aptitude_1[2], aptitude_1[3], aptitude_2[1], aptitude_2[2], aptitude_2[3], difficulty[1], difficulty[2], difficulty[3], difficulty[4], difficulty[5]
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, n_steps, nom_step_size, step_size

Summary Statistics
     parameters      mean       std   naive_se      mcse       ess      rhat 
         Symbol   Float64   Float64    Float64   Float64   Float64   Float64 

  aptitude_1[1]    4.1125    2.8127     0.0889    0.7651   12.7687    1.0016
  aptitude_1[2]    2.0198    2.9273     0.0926    0.7668    4.0284    1.4551
  aptitude_1[3]   -4.7104    2.8762     0.0910    0.8053    6.7425    1.0645
  aptitude_2[1]    3.2786    3.0173     0.0954    0.7514    5.4254    1.2778
  aptitude_2[2]   -0.6002    2.5990     0

## Test-taker Fatigue

Imagine the test is several hours long. The test taker is pretty likely to perform differently (let's assume worse) by the end of the test, and that fatigue factor is probably pretty specific to the person. Thus, for the $i^{th}$ question we introduce a linear penalty as a first stab at the idea:

$$\begin{aligned}
\alpha_s &\sim \mathrm{Normal}(0,5)\\
\phi_s &\sim \mathrm{LogNormal}(0,1)\\
\gamma_q &\sim \mathrm{Normal}(0,5)\\
\beta_{s, q, i} &= \mathrm{logit}^{-1}(\alpha_s - \gamma_q - i\phi_s)\\
\mathrm{correct_{s,q,i}} &\sim \mathrm{Bernoulli}(\beta_{s,q,i})\\
\end{aligned}$$

In [43]:
question_seq = [1,2,3,4, 1,2,3,4, 1,2,3,4]
@model function irt_fatigue(
        correct::Array{Bool}, student::Array{Int64}, 
        question::Array{Int64}, question_seq::Array{Int64}
    )
    aptitude = Vector(undef, maximum(student))
    wimpiness = Vector(undef, maximum(student))
    difficulty = Vector(undef, maximum(question))
    
    # priors
    for i in 1:length(aptitude)
        aptitude[i] ~ Normal(0,5)
        wimpiness[i] ~ LogNormal(0,2)
    end
    for i in 1:length(difficulty)
        difficulty[i] ~ Normal(0,5)
    end        
    
    β = Vector(undef, length(correct))
    for i in 1:length(correct)
        β[i] = aptitude[student[i]] - difficulty[question[i]] - wimpiness[student[i]] * i * question_seq[i]
        correct[i] ~ Bernoulli(inv_logit(β[i]))
    end
end;

In [44]:
#collapse
irt_fatigue_ch = sample(
    irt_fatigue(correct, student, question, question_seq), 
    HMC(ϵ, τ), iterations, 
    progress=true, drop_warmup=true)

Sampling: 100%|█████████████████████████████████████████| Time: 0:00:00


Chains MCMC chain (1000×20×1 Array{Float64,3}):

Iterations        = 1:1000
Thinning interval = 1
Chains            = 1
Samples per chain = 1000
parameters        = aptitude[1], aptitude[2], aptitude[3], difficulty[1], difficulty[2], difficulty[3], difficulty[4], difficulty[5], wimpiness[1], wimpiness[2], wimpiness[3]
internals         = acceptance_rate, hamiltonian_energy, hamiltonian_energy_error, is_accept, log_density, lp, n_steps, nom_step_size, step_size

Summary Statistics
     parameters      mean       std   naive_se      mcse       ess      rhat 
         Symbol   Float64   Float64    Float64   Float64   Float64   Float64 

    aptitude[1]    5.5689    3.2672     0.1033    0.9661    5.5541    1.0900
    aptitude[2]   -0.6428    2.2023     0.0696    0.5558    5.5129    1.2839
    aptitude[3]   -2.0090    2.7835     0.0880    0.7741    6.2247    1.1309
  difficulty[1]   -7.5951    3.1442     0.0994    0.8897   10.2621    1.0804
  difficulty[2]   -3.5479    3.0344     0.0960    

# Note on the model specifications
 
You might be wondering "where did these prior values come from?" or "how did Brad choose these distributions? Why Normal instead of, I dunno, t?". Good questions! The answer is I didn't think too hard and just wrote down the first thing that seemed reasonable :sweat_smile: either in terms of the values ($\mathrm{logit}^{-1}(5)$ is a very high - 99-ish% - but not insurmountable level of confidence) or theoretical properties (basically, choose a simple distribution with the right domain and range).

Perhaps you're also wondering "what's up with the random mixing in of Greek letters?" You got me there.

# Comparison to Collaborative Filtering

Item Response may seem very similar to collaborative filtering, so it's worth highlighting the differences.

Collaborative filtering aims to complete a sparsely determined preferences/ratings matrix of consumer - item scores (e.g. "User 513 gave product 149 3.5 :star:s") $M$. A common approach is alternating least squares which iteratively factors the matrix into a product-feature matrix $P$ and a customer-preference matrix $C$. The goal is to create these so that their product "accurately completes" $M$, ie if $CP = \overline{M}$ then the difference $M - \overline{M}$ is small wherever we have entries for $M$ (remember, $M$ is incomplete).

A key fact is that the matrix $\overline{M}$ (the list of recommendations) is the important output here, and the factors $C$ and $P$ are intriguing but not critical. This is different from Item Response where the background variables describing difficulty and aptitude for each question, student are the primary desired outputs (but we could infer $P(\mathrm{correct} | \mathrm{student\_id}, \mathrm{question\_id})$ for unseen pairings!).

The other distinction worth mentioning is that the IR models have enormous flexibility in how they inform the probability of success, as we've seen above. Collaborative filtering, at least with ALS, is just optimizing a matrix factorization task. Since $\overline{M} = CP$, the user-product score can only be the dot product of the product-feature vector and the customer-preference vector, it attempts to encode "how well do the consumer preferences overlap with the product features." It does not lend itself to any extensions to capture more domain-specific nuance as we did here.